In [2]:
import pandas as pd
import numpy as np

GAMR

In [55]:
participants = ["Participant_1", "Participant_2", "Participant_3"]
gest_type = ["icon-GA", "gesture-unit", "and", "deixis-GA", "emblem-GA"]
arg0 = ["participant_1", "participant_2", "participant_3"]
arg1 = list(set(['thumbs_up', 'scale', 'wobble', 'idea', 'tablet', 'researcher', 'paper', 'together', 'type', 'stack', 'hold', 'row', 'participant_3', 'string', 'stop', 'look', 'even', 'worksheet', 'yellow_block', 'writing', 'wobble', 'high', 'green_block', 'larger', 'location', 'low', 'computer', 'list', 'fingers', 'blue_block', 'clap', 'agree', 'phone', 'remove', 'blue_block', 'put', 'purple_block', 'participant_1', 'brown_block', 'range', 'counting', 'add', 'fall', 'hand', 'drop', 'clap', 'cup', 'papert', 'wait', 'yellow_block', 'participant_2', 'blocks', 'group', 'up', 'green_block', 'narrow', 'done', 'previous', 'idea', 'move', 'circle', 'brown _block', 'typing', 'hold', 'maybe', 'approximate', 'red_block', 'no', 'swipe direction', 'notebook_scale', 'purple_block', 'yes', 'box', 'left', 'throw', 'block']))
arg2 = ["group", "participant_1", "participant_2", "participant_3", "researcher"]

total_len = len(gest_type) + len(arg0) + len(arg1) + len(arg2)

In [63]:
total_len

81

In [86]:
for group in range(1,11):
    # print(group)
    df_final = pd.DataFrame()
    
    gamr = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_GAMR_Oracle.csv").fillna('')

    utt_end = gamr.iloc[-1].Utterance

    for i in range(utt_end+1):
        parts = [[0]*total_len, [0]*total_len, [0]*total_len]

        df = gamr.loc[gamr["Utterance"] == i]
        for _, row in df.iterrows():
            for p_i, p in enumerate(participants):
                val = row.loc[p]


                gest_tmp = [0]*len(gest_type)
                for i, g in enumerate(gest_type):
                    if g in val:
                        gest_tmp[i] = 1


                arg0_tmp = [0]*len(arg0)
                arg0_start = val.find("ARG0")
                arg0_end = arg0_start + val[arg0_start:].find(")")+1
                if arg0_start != 0 and arg0_end-arg0_start > 0:
                    v = val[arg0_start:arg0_end]
                    if "center" in v or "2" in v:
                        arg0_tmp[1] = 1
                    elif "left" in v or "1" in v:
                        arg0_tmp[0] = 1
                    elif "right" in v or "3" in v:
                        arg0_tmp[2] = 1


                arg1_tmp = [0]*len(arg1)
                arg1_start = val.find(":ARG1") + 1
                arg1_end = arg1_start + val[arg1_start:].find(")")+1
                if arg1_start != 0 and arg1_end-arg1_start > 0:
                    v = val[arg1_start:arg1_end]
                    for i, opt in enumerate(arg1):
                        if opt in v:
                            arg1_tmp[i] = 1


                arg2_tmp = [0]*len(arg2)
                arg2_start = val.find("ARG2")
                arg2_end = arg2_start + val[arg2_start:].find(")")+1
                if arg2_start != 0 and arg2_end-arg2_start > 0:
                    v = val[arg2_start:arg2_end]
                    if v == "r":
                        arg2_tmp[arg2.index("r")] = 1
                    elif v == "g":
                        arg2_tmp[arg2.index("g")] = 1
                    elif "2" in v:
                        arg2_tmp[arg2.index("participant_2")] = 1
                    elif "1" in v:
                        arg2_tmp[arg2.index("participant_1")] = 1
                    elif "3" in v:
                        arg2_tmp[arg2.index("participant_3")] = 1
                

                parts[p_i] = [x + y for x, y in zip(parts[p_i], gest_tmp + arg0_tmp + arg1_tmp + arg2_tmp)]
        tensor_df = pd.DataFrame(np.array(parts).reshape((1, -1)))
        df_final = pd.concat((df_final, tensor_df))
    df_final.to_csv(f"Group_{group:02d}/GROUP_{group:02d}_GAMR_Oracle_vectors.csv", index=False, header=None)


Action

In [113]:
action_types = ['put', 'lift'] # check with if action type IN not == (some typos like llift), and use lower()obj1 = 
obj1 = ["red", "blue", "green", "mystery", "purple", "yellow"] # same
preposition = ["in", "on"]
obj2 = ["phone", "table", "floor", "lefthand", "righthand", "leftblock", "rightblock", "leftscale", "rightscale", "mystery", "red", "blue", "green", "purple", "yellow"]
object_action_types = ["right", "left", "zero"]

total_len = len(action_types) + len(obj1) + len(preposition) + len(obj2)

In [114]:
total_len

25

In [119]:
total_len * 3 + len(object_action_types)

78

In [121]:
for group in range(1, 11):
    # print(group)
    df_final = pd.DataFrame()
    action_df = pd.read_csv(f"Group_{group:02d}/Group_{group:02d}_Actions_Oracle.csv").fillna('')
    action_cols = action_df.columns.difference(["Start", "End", "Utterance"])


    for i in range(len(action_df)):
        p = [[0]*total_len, [0]*total_len, [0]*total_len]
        for act in action_cols:
            if "object" not in act.lower():
                    action = action_df.iloc[i][act]
                    if action:
                        for p_i in range(1, 4):
                            if str(p_i) in action[-2:]:
                                action_types_tmp = [0]*len(action_types)
                                obj1_tmp = [0]*len(obj1)
                                obj2_tmp = [0]*len(obj2)
                                preposition_tmp = [0]*len(preposition)
                                
                                for i_a, a in enumerate(action_types):
                                    if a in action[:action.find('(')].lower():
                                        action_types_tmp[i_a] += 1
                                
                                for i_a, a in enumerate(obj1):
                                    if action.find(',') == -1:
                                        if a in action[action.find('(')+1: action.find(')')].lower():
                                            obj1_tmp[i_a] += 1
                                    else:
                                        if a in action[action.find('(')+1: action.find(',')].lower():
                                            obj1_tmp[i_a] += 1
                                
                                for i_a, a in enumerate(preposition):
                                    if "put" in action.lower():
                                        if action.find(',') != 1:
                                            if a in action[action.find(',')+1:action[action.find(','):].find('(') + action.find(',')].lower():
                                                preposition_tmp[i_a] += 1
                                
                                for i_a, a in enumerate(obj2):
                                    if "put" in action.lower():
                                        if action.find(',') != -1:
                                            idx = max(action.find('in'), action.find('on'))
                                            if a in action[idx+3:][:action[idx+3:].find(')')].lower():
                                                obj2_tmp[i_a] += 1
                                
                                p[p_i - 1] = [x + y for x, y in zip(p[p_i - 1], action_types_tmp + obj1_tmp + preposition_tmp + obj2_tmp)]


            else:
                object_action_types_tmp = [0]*len(object_action_types)
                if action:
                    idx = action.find("lean")
                    for i_a, a in enumerate(object_action_types):
                        if a in action[idx:][action[idx:].find('('):].lower():
                            object_action_types_tmp[i_a] = 1
        tensor_df = pd.DataFrame(np.array(p[0] + p[1] + p[2] + object_action_types_tmp).reshape((1, -1)))
        df_final = pd.concat((df_final, tensor_df))
    df_final.to_csv(f"Group_{group:02d}/GROUP_{group:02d}_Actions_Oracle_vectors.csv", index=False, header=None)